# Homework information


**Students:** Черкашина Елизавета Андреевна, Мешков Михаил Алексеевич

**Master’s programme:** Data Analytics and Social Statistics

**Task:**  Module 2. Hard skills project № 4 `Text analysis'

**Date:** 17.12.2024  

## Information about environment versions

- Python 3.12.7
- Pandas 2.2.3
- Matplotlib 3.9.2
- spacy/flair/ stanza/ nltk - *после того как определимся с пакетом*
- Scikit-learn  1.5.1


# Описание задачи (как сделаем можно убрать)

## Tasks
**To receive 70 / 100 points (basic questions):**
1. Load "text" and "target" columns to pandas dataframe. Pick random 1000 lines from the dataframe. Compute average text length, and dictionary size.
2. Find POSITIVE and NEGATIVE texts using flair or spacy. Add sentiment classification result to the dataframe. Supply this section with sentiment label distribution chart and discussion.
3. Compare spacy (or flair) results with the "target" column, build confusion matrix and discuss it.
4. Find all PERSONs and ORGANISATIONs in POSITIVE texts and add them to your dataframe. If there are multiple entities inside one text create separate lines for each entity.
5. Find the most frequent PERSON and ORGANISATION in positive texts.
5. Find the most frequent PERSON and ORGANISATION in positive texts.

**To receive additional 30 / 100 points (advanced questions):**

6. Cluster NEGATIVE texts using k-means clustering. Use tf-idf function to rank important phrases inside each document.
7. Describe each NEGATIVE cluster with a list of key phrases or summary.

## Submission format
You should submit your project as the python notebook (.ipynb) file and supplementary pdf file, generated from the python notebook. All questions of the Task section should be covered with code and markdown comments that are supposed to give a brief idea about what is happening in the code below and brief results discussion. Your report should be no longer than 5 pages including pictures.

* Each section (1-7) should be supported with introduction and conclusion/discussion markdown text to be considered done.
* The code cells should be executed and do not contain compilation and logical mistakes, there should not be unnecessary code and unused libraries inside the notebook.

## Tips and tricks

You can use materials from practical sections of Module 5 “Contemporary Text Analysis”.
In addition to that, below you can find the function to create a confusion matrix given the predicted and target columns in dataframe:
``` python
import pandas as pd
def confusion_matrix(df: pd.DataFrame, col1: str, col2: str): return (df
.groupby([col1, col2]) .size()
.unstack(fill_value=0)
) ```

# Code

## Input block

In [1]:
# import librares
#import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [2]:
# download latest version dataset
#path = kagglehub.dataset_download("kazanova/sentiment140")

#a = print("Path to dataset files:", path)

**Dataset description** from https://www.kaggle.com/datasets/kazanova/sentiment140/data

**Context**

This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment.

**Content**

It contains the following 6 fields:

1. target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

2. ids: The id of the tweet ( 2087)

3. date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

4. flag: The query (lyx). If there is no query, then this value is NO_QUERY.

5. user: the user that tweeted (robotickilldozr)

6. text: the text of the tweet (Lyx is cool)



In [3]:
# import df
df = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                 encoding='latin-1', 
                 names = ["target", "ids", "date", "flag", "user", "text"])

In [4]:
# look at df
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
